In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/simple-dialogs-for-chatbot/dialogs.txt


In [2]:
import spacy
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
import pandas as pd
import numpy as np

In [3]:
!spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.3 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
nlp = spacy.load("en_core_web_lg")

In [5]:
import unicodedata
import re

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def clean_text(text):
    text = unicode_to_ascii(text.lower().strip())
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"\r", "", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation)) 
    text = re.sub("(\\W)"," ",text) 
    text = re.sub('\S*\d\S*\s*','', text)
    return text

In [8]:
import string

df = pd.read_csv('../input/simple-dialogs-for-chatbot/dialogs.txt',sep='\t')

df

,"hi, how are you doing?",i'm fine. how about yourself?
0,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
1,i'm pretty good. thanks for asking.,no problem. so how have you been?
2,no problem. so how have you been?,i've been great. what about you?
3,i've been great. what about you?,i've been good. i'm in school right now.
4,i've been good. i'm in school right now.,what school do you go to?
...,...,...
3719,that's a good question. maybe it's not old age.,are you right-handed?
3720,are you right-handed?,yes. all my life.
3721,yes. all my life.,you're wearing out your right hand. stop using...
3722,you're wearing out your right hand. stop using...,but i do all my writing with my right hand.


In [9]:
def remove_punctuation(sentence):
    return sentence.translate(str.maketrans('', '', string.punctuation))

df.rename(columns={"i'm fine. how about yourself?": 'sentence'}, inplace=True)
df.rename(columns={"hi, how are you doing?": 'question'}, inplace=True)

df['sentence'] = df['sentence'].apply(clean_text)
df['sentence'] = df['sentence'].apply(remove_punctuation)

df['question'] = df['question'].apply(clean_text)
df['question'] = df['question'].apply(remove_punctuation)

df

,question,sentence
0,i am fine how about yourself,i am pretty good thanks for asking
1,i am pretty good thanks for asking,no problem so how have you been
2,no problem so how have you been,i have been great what about you
3,i have been great what about you,i have been good i am in school right now
4,i have been good i am in school right now,what school do you go to
...,...,...
3719,that is a good question maybe it is not old age,are you righthanded
3720,are you righthanded,yes all my life
3721,yes all my life,you are wearing out your right hand stop using...
3722,you are wearing out your right hand stop using...,but i do all my writing with my right hand


In [ ]:
def extract_last_word(sentence):
    words = sentence.split()
    if len(words) > 0:
        return words[-1]
    else:
        return None

df['last_word'] = df['sentence'].apply(extract_last_word)

def remove_last_word(sentence):
    words = sentence.split()
    if len(words) > 1:
        return ' '.join(words[:-1])
    else:
        return ''
    
df['cut_sentence'] = df['sentence'].apply(remove_last_word)
df

In [10]:
spacy_vocab = set([word.text for word in nlp.vocab])

y_sequences = df['sentence'].apply(lambda x: [word.text for word in nlp(x)]).tolist()

X = np.array([nlp(sentence).vector for sentence in df['question']])
y = np.zeros((len(df), len(spacy_vocab)))

In [11]:
word_to_index = {word: i for i, word in enumerate(spacy_vocab)}

for i, words in enumerate(y_sequences):
    for word in words:
        # Get the corresponding index for the word
        word_index = word_to_index.get(word, None)

        if word_index is not None:
            y[i, word_index] = 1

In [15]:
from keras.layers import TimeDistributed

model = Sequential()
model.add(LSTM(units=50, activation='tanh', input_shape=(1, X.shape[1]), return_sequences=True))
model.add(TimeDistributed(Dense(units=len(spacy_vocab), activation='softmax')))


In [13]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1081, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1139, in compute_loss
        return self.compiled_loss(
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    

In [ ]:
test_sentence = 'I want to speak english'
test_vector = nlp(test_sentence).vector
test_vector = np.array(test_vector).reshape(1, -1) 

predicted_probs = model.predict(test_vector, verbose=0)
predicted_word_index = np.argmax(predicted_probs)

# If you have a mapping from index to word, you can use it to get the actual word
predicted_word = list(spacy_vocab)[predicted_word_index]

print(f"Test sentence: {test_sentence}")
print(f"reply: {predicted_word}")

In [ ]:
predicted_word_index